In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/02/28 18:04:53 WARN Utils: Your hostname, Shumengs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.15 instead (on interface en0)
24/02/28 18:04:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/28 18:04:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#### Q1

In [2]:
spark.version

'3.3.4'

#### Q2

In [24]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [25]:
df = spark.read \
    .option("header", "true") \
    .schema(schema)\
    .csv('fhv_tripdata_2019-10.csv.gz')

In [28]:
df = df.repartition(6)
df.write.parquet('HW/fhv_2019_10/')

#### Q3

In [31]:
from pyspark.sql.functions import col

In [33]:
df.filter(col("pickup_datetime").cast("date") == '2019-10-15').count()

62610

#### Q4

In [34]:
df.registerTempTable('fhv_data')

/usr/local/spark/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [42]:
df_q4 = spark.sql("""
SELECT 
    *,
    TIMESTAMPDIFF(SECOND, pickup_datetime, dropoff_datetime) / 3600 as duration_hour
FROM
    fhv_data
ORDER BY TIMESTAMPDIFF(SECOND, pickup_datetime, dropoff_datetime) / 3600 DESC

""")

In [45]:
df_q4.select(['duration_hour']).show(1)

+-------------+
|duration_hour|
+-------------+
|     631152.5|
+-------------+
only showing top 1 row



#### Q6

In [46]:
df_zone = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [47]:
df_zone.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [58]:
df_join = df.join(df_zone, df.PULocationID == df_zone.LocationID)

In [63]:
df_join = df_join.drop('LocationID','Borough','service_zone')
df_join.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|    Zone|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+--------+
|              B02546|2019-10-20 22:07:05|2019-10-20 22:09:27|         264|         235|   null|                B02546|      NV|
|              B00310|2019-10-31 11:17:27|2019-10-31 11:24:59|         264|         212|   null|                B00310|      NV|
|              B00647|2019-10-31 07:59:07|2019-10-31 08:02:31|         264|          78|   null|                B00647|      NV|
|              B01626|2019-10-31 09:40:51|2019-10-31 09:44:14|         264|         130|   null|                B01626|      NV|
|     B01711         |2019-10-31 13:13:45|2019-10-31 13:54:57|          92|         173|   null| 

In [64]:
df_join.registerTempTable('hw_q6')

In [73]:
df_q6 = spark.sql("""
SELECT 
 
    zone,
    COUNT(1) AS number_records
FROM
    hw_q6

GROUP BY
    1
ORDER BY 2 ASC
LIMIT 1
""")

In [74]:
df_q6.show()

+-----------+--------------+
|       zone|number_records|
+-----------+--------------+
|Jamaica Bay|             1|
+-----------+--------------+

